In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import csv
import operator
from operator import itemgetter

# for testing the label file can contain the bounding box location in pixels (= 1) instead of 
# percentages of the entire image. For use in YOLO, set it = 0. 
label_output_in_pixels = 0  
# create an empty label file for tiles containing no objects?
save_empty_tiles = 0

# YOLO uses 416x416 pixel images by default
desired_output_width = 416
desired_output_height = 416
overlap = 0.10 # desired overlap of images - will be changed to ensure the desired ouput width & height

# loop through all .JPG files in directory.
images = [('imgs/' + f) for f in os.listdir('imgs') if (os.path.isfile(os.path.join( 'imgs', f)) and f.endswith('.JPG'))]

# loop through all images found
for input_file_name in images:
    # load image
    img = cv2.imread(input_file_name)

    # label file in directory 'labels_pixels' and the file name <image name without .JPG extension> + '_labels.txt'
    label_file = 'label_pixels/' + input_file_name.split('imgs/')[1].split('.JPG')[0] + '_labels.txt'

    # calculate number of output image tiles
    input_height, input_width = img.shape[0:2]
    print("Input height:", input_height, "\tInput width:",input_width)
    
    # calculate the number of tiles we need along each dimension
    horizontal_tiles = round((input_width - desired_output_width) / (desired_output_width * (1 - overlap)))
    vertical_tiles = round((input_height - desired_output_height) / (desired_output_height * (1 - overlap)))

    # calculate the amount the overlap needs to be increased/ decreased to enable about the 
    # right number of tiles without going beyond the image boundaries
    tile_width_exact_fit = (input_width - desired_output_width) / horizontal_tiles
    tile_height_exact_fit = (input_height - desired_output_height) / vertical_tiles

    overlap_adjusted_horizontal = (desired_output_width - tile_width_exact_fit) / desired_output_width
    overlap_adjusted_vertical = (desired_output_height - tile_height_exact_fit) / desired_output_height

    #print('horizontal tiles:', horizontal_tiles)
    #print('vertical tiles:', vertical_tiles)
    #print('overlap_adjusted_horizontal:',overlap_adjusted_horizontal)
    #print('overlap_adjusted_vertical', overlap_adjusted_vertical)

    output_width = desired_output_width 
    output_height = desired_output_height 

    print('Creating', horizontal_tiles, '*', vertical_tiles, '=', horizontal_tiles*vertical_tiles, 'output image tiles.')
    print('Tile width:', output_width)
    print('Tile hight:', output_height)
    print('Horizontal Overlap:', overlap_adjusted_horizontal)
    print('Vertical Overlap:', overlap_adjusted_vertical)

    # parse label file for image
    # input file format: <label> <x> <y> <width> <height> in pixels
    img_labels = open(label_file, 'r')
    temp_labels = img_labels.read().strip().split('\n')

    labels = []
    for element in temp_labels:
        labels.append(element.split(' '))

    # convert list items to ints
    labels = [[int(float(x)) for x in y] for y in labels] 

    # sort the list by location of the item in x direction
    labels = sorted(tuple(labels), key = itemgetter(1))
    #print("Labels for current tile:", labels)
    
    # pre-allocate image matrix
    tile = np.zeros((output_width, output_height))

    # create output folder if it doesn't already exist
    if not os.path.exists('imgs/' + input_file_name.split('/')[1].split('.')[0]):
        print("os.mkdir('imgs/' + input_file_name.split('/')[1].split('.')[0])")
    if not os.path.exists('imgs/with_obj'):    
        os.mkdir('imgs/with_obj/')
    print('Saving images to', 'imgs/' + input_file_name.split('/')[1].split('.')[0])

    # loop through original image and save each tile
    for i in range(0, horizontal_tiles + 1): 
            for j in range(0, vertical_tiles + 1):
                    #print(i, ':', i * output_width - i * output_width * overlap)
                    print('Processing file', input_file_name.split('/')[1])
                    
                    # calculate tile begin in x and y direction
                    tile_begin_x = math.floor(i * output_width - i * output_width * overlap_adjusted_horizontal)
                    tile_end_x = math.floor(tile_begin_x + output_width)
                    tile_begin_y = math.floor(j * output_height - j * output_height * overlap_adjusted_vertical)
                    tile_end_y = math.floor(tile_begin_y + output_height)
                    #print('(', i,',', j, ')', 'x:', tile_begin_x, tile_end_x, 'y:', tile_begin_y, tile_end_y)

                    # draw the tile locations for testing
                    #cv2.line(img, (tile_begin_x, 0), (tile_begin_x, input_height), (0, 0, 255), 10)
                    #cv2.line(img, (0, tile_begin_y), (input_width, tile_begin_y), (0, 0, 255), 10)
                    
                    tile = img[tile_begin_y:tile_end_y, tile_begin_x:tile_end_x]
                    
                    # check for object in the image and save labels to file, if any exist
                    # pre-allocate temporary list of labels for current tile
                    tile_labels = []
                    for element in labels:
                        # check if the element is inside the current tile.
                        if (element[1] > tile_begin_x) and (element[1] < tile_end_x) and (element[2] > tile_begin_y) and (element[2] < tile_end_y):
                            tile_labels.append(element)

                            # subtract the location of the tile from the object location to make sure it's relative to the tile position
                            tile_labels[-1] = list(map(operator.sub, tile_labels[-1], [0, tile_begin_x, tile_begin_y, 0, 0]))
                            print(tile_labels[-1])
                            # draw bounding box on image for testing
                            rectangle_offset = 0  # optional parameter for potentially drawings bboxes more clearly
                            #cv2.rectangle(tile, (tile_labels[-1][1] - rectangle_offset, tile_labels[-1][2] - rectangle_offset), (tile_labels[-1][1] + tile_labels[-1][3] + rectangle_offset, tile_labels[-1][2] + tile_labels[-1][4] + rectangle_offset), (255, 0, 0), 1)
                                                       
                            print('Object at x:', tile_labels[-1][1], 'y:', tile_labels[-1][2], 'found in tile', '(', i, ',', j, ').')
                            
                        
                    if tile_labels:
                        # save images with an object in a separate folder
                        output_file_name = r'imgs/with_obj/' + input_file_name.split('/')[1].split('.')[0] + '_tile' + str(i) + '-' + str(j) + '.jpg'                                  
                        cv2.imwrite(output_file_name, tile)

                        #print('Saving tile (', i, ',', j, ') to ', output_file_name)
                    else:
                        output_file_name = r'imgs/empty/' + input_file_name.split('/')[1].split('.')[0] + '_tile' + str(i) + '-' + str(j) + '.jpg'                    
                            
                        if save_empty_tiles == 1:
                            cv2.imwrite(output_file_name, tile)

                    # save the labels to a file
                    with open(output_file_name.split('.jpg')[0] + '.txt', 'w') as tile_label_file:
                        if tile_labels:  # if tile_labels not empty
                            #print('Working on', tile_labels)
                            # make sure the labels don't contain a 0.0 (replace by 1e-9) since darknet causes problems
                            # otherwise. Also, clamp high values to 1.0
                            if label_output_in_pixels == 1:
                                for obj in tile_labels:
                                    obj[1] = (obj[1] + obj[3] / 2)  
                                    obj[1] = np.clip(obj[1], 0, desired_output_width)
                                    obj[2] = (obj[2] + obj[4] / 2) 
                                    obj[2] = np.clip(obj[2], 0, desired_output_height)
                                    obj[3] = obj[3] 
                                    obj[3] = np.clip(obj[3], 0, desired_output_width)
                                    obj[4] = obj[4] 
                                    obj[4] = np.clip(obj[4], 0, desired_output_height)
                                    
                            else:
                                for obj in tile_labels:
                                    obj[1] = (obj[1] + obj[3] / 2) / desired_output_width 
                                    obj[2] = (obj[2] + obj[4] / 2) / desired_output_height
                                    obj[3] = obj[3] / desired_output_width
                                    obj[4] = obj[4] / desired_output_height
                                    obj = np.clip(obj, 1e-9, 1)
                            
                        writer = csv.writer(tile_label_file, delimiter =' ')
                        writer.writerows(tile_labels) 
                

    # plot current tile, save image
    #plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #cv2.imwrite('imgcv.jpg', img) # .png for higher quality (huge though)
    #plt.show()

